In [1]:
using FluxArchitectures, Plots

poollength = 10
horizon = 15
datalength = 1000
input, target = get_data(:exchange_rate, poollength, datalength, horizon) |> gpu

([0.062407967 0.0353257 … 0.1238919 0.09095394; -0.14302583 -0.14921565 … 0.21910584 0.14172706; … ; -0.53270185 -0.5240285 … -0.37224257 -0.3939262; -1.7405509 -1.7588272 … -1.7087181 -1.7087181;;;; 0.0353257 0.07119151 … 0.09095394 0.10778896; -0.14921565 -0.029743362 … 0.14172706 0.09530037; … ; -0.5240285 -0.49540597 … -0.3939262 -0.36790615; -1.7588272 -1.7305316 … -1.7087181 -1.6935327;;;; 0.07119151 0.06606788 … 0.10778896 0.1253556; -0.029743362 0.017921582 … 0.09530037 0.064349; … ; -0.49540597 -0.49800792 … -0.36790615 -0.35055894; -1.7305316 -1.7604929 … -1.6935327 -1.6969123;;;; … ;;;; -0.46825543 -0.47191533 … -0.48069844 -0.52900726; -0.87533563 -0.8629553 … -0.85181236 -0.9230006; … ; -0.6888242 -0.70183444 … -0.7425997 -0.76428336; -0.47123808 -0.45583472 … -0.41464683 -0.39265272;;;; -0.47191533 -0.4704512 … -0.52900726 -0.57072836; -0.8629553 -0.8765739 … -0.9230006 -0.9397142; … ; -0.70183444 -0.7079058 … -0.76428336 -0.8249973; -0.45583472 -0.46546757 … -0.39265272 

In [4]:
input

8×10×1×1000 CUDA.CuArray{Float32, 4, CUDA.Mem.DeviceBuffer}:
[:, :, 1, 1] =
  0.062408   0.0353257   0.0711915  …   0.102665   0.123892   0.0909539
 -0.143026  -0.149216   -0.0297434      0.14049    0.219106   0.141727
  0.340551   0.33548     0.334848       0.362801   0.362169   0.312129
 -1.26693   -1.27098    -1.18218       -1.11631   -1.06012   -1.11761
  2.84981    2.84981     2.84981        2.84981    2.84981    2.84981
 -1.70661   -1.68957    -1.61326    …  -1.67935   -1.67935   -1.68208
 -0.532702  -0.524028   -0.495406      -0.399131  -0.372243  -0.393926
 -1.74055   -1.75883    -1.73053       -1.71545   -1.70872   -1.70872

[:, :, 1, 2] =
  0.0353257   0.0711915   0.0660679  …   0.123892   0.0909539   0.107789
 -0.149216   -0.0297434   0.0179216      0.219106   0.141727    0.0953004
  0.33548     0.334848    0.343719       0.362169   0.312129    0.31843
 -1.27098    -1.18218    -1.16969       -1.06012   -1.11761    -1.09221
  2.84981     2.84981     2.84981        2.84981    

In [34]:
inputsize = size(input, 1)
convlayersize = 10
recurlayersize = 10
skiplength = 120
model = LSTnet(inputsize, convlayersize, recurlayersize, poollength, skiplength, init=Flux.zeros32, initW=Flux.zeros32) |> gpu

LSTnet(8, 10, 10, 10, 120)

In [28]:
function loss(x, y)
    Flux.ChainRulesCore.ignore_derivatives() do
        Flux.reset!(model)
    end
    return Flux.mse(model(x), y')
end

loss (generic function with 1 method)

In [31]:
cb = function ()
    Flux.reset!(model)
    pred = model(input)' |> cpu
    Flux.reset!(model)
    p1 = plot(pred, label="Predict")
    p1 = plot!(cpu(target), label="Data", title="Loss $(loss(input, target))")
    display(plot(p1))
end

#19 (generic function with 1 method)

In [ ]:
@info "Start loss" loss = loss(input, target)
@info "Starting training"
Flux.train!(loss, Flux.params(model),Iterators.repeated((input, target), 20),
            Adam(0.01), cb=cb)
@info "Final loss" loss = loss(input, target)